In [ ]:
!pip install -U sentence-transformers

In [2]:
import pandas as pd
from langdetect import detect

In [1]:
#Mounting account's drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#Reading the data from file
df = pd.read_csv("/content/drive/MyDrive/CSCI6364Proj/matched_lyrics.csv")

In [5]:
df.head()

,track_id,title,artist_name,Lyrics
0,TRMMMQN128F4238509,Raspberry Beret (LP Version),Prince & The Revolution,"One, two\nOne, two, three, uh\n\nYeah\n\nI was..."
1,TRMMWTG128F4283F07,You Needed Me,Roger Whittaker,I cried a tear: you wiped it dry.\nI was confu...
2,TRMMZRE128F42B799E,Sauver L'Amour,Daniel Balavoine,Partir effacer sur le Gange\nLa douleur\nPouvo...
3,TRMMZOT128F149E9EE,Prayee,The Chantels,NaN
4,TRMMIZZ128F4289298,It Keep Rainin' (Tears From My Eyes) (Radio Edit),Bitty McLean,Pretty fallin' tears shining (Yes)\n(What's th...


In [6]:
df['Lyrics'].isnull().sum()

418

In [7]:
df.shape

(12447, 4)

In [8]:
df.dropna(axis = 0, inplace = True)

In [9]:
df.shape

(12029, 4)

In [10]:
df['Lyrics'].isnull().sum()

0

In [11]:
df['Lyrics'][2905]

"Colore le monde\nSans feutres, sans épreuves ni bombes\nIndolore les murs\nEt coule dans le fleuve la facture\nEn monnaie de singe\nFraîche, blanche comme le linge\nÀ jamais répand\nDu fard sur les hommes, car le temps\n\nColore la foule, colore mes veines\nChaque jour il me révèle\nEn chair de poule, en bleu du ciel\nEt la foule sort de mes veines\nOui comme le temps est un ami\nIl colore mon pays\n\nDes dieux, mes soeurs\nJ'en vois de toutes les couleurs\nPour des cieux, mes frères\n...\n\n******* This Lyrics is NOT for Commercial use *******"

In [12]:
lyrics_list = df.Lyrics.values.tolist()
len(lyrics_list)

12029

In [40]:
track_list = df.track_id.values.tolist()
len(track_list)

12029

In [ ]:
count = 0
l_list = []
t_list = []
for i in range (0,len(lyrics_list)):
  count=count+1
  print(count)
  try:
    if (detect(lyrics_list[i])=='en'):
      final = lyrics_list[i].replace('\n','')
      final = final.replace('******* This Lyrics is NOT for Commercial use *******','')
      l_list.append(final)
      t_list.append(track_list[i])
    else:
      continue
  except:
    continue
len(l_list)

In [25]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [ ]:
#Encoding:
sen_embeddings = []
count = 0
for item in l_list:
  count=count+1
  print(count)
  se = model.encode(item)
  sen_embeddings.append(se)

In [17]:
#Store sentences & embeddings on disc

import pickle
with open('/content/drive/MyDrive/CSCI6364Proj/embeddingscomp2VF.pkl', "wb") as fOut:
    pickle.dump({'sentences': l_list[:10], 'embeddings': sen_embeddings}, fOut, protocol=pickle.HIGHEST_PROTOCOL)

In [33]:
from sklearn.metrics.pairwise import cosine_similarity
#calculating cosine similarity for first two songs
ans = cosine_similarity([tok],sen_embeddings)

In [6]:
#Load sentences & embeddings from disc
import pickle
with open('/content/drive/MyDrive/CSCI6364Proj/embeddingscomp2VF.pkl', "rb") as fIn:
    stored_data = pickle.load(fIn)
    stored_sentences = stored_data['sentences']
    stored_embeddings = stored_data['embeddings']

In [26]:
tok = model.encode("I love you like a love song yeah i don't know what it mean i think you could be in the corner take a chance to you and i know that you're in any way i can do i don't want to go like you don't know don't you wanna give me that i won't take a little love from your life i want to make you mine i want to break my life i want to break you all i want to make you go ooh")

In [63]:
import csv
with open('/content/drive/MyDrive/CSCI6364Proj/t_list.csv', 'w', newline='') as myfile:
     wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
     wr.writerow(t_list)

## Usage


In [2]:
#Mounting account's drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#load the model
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [11]:
#convert given text into embedding
text="the sky is blue and the home is on your back kicked in the street take a chance on a walk on the borderline on the borderline oh oh no ma and the night look out of the door and the night of the fantasy yeah i'm gonna get you a rocket man on a man watch out man on the prowl man you know what a gun for a big man with a big man with a big man with a big man"
tok = model.encode(text)

In [12]:
#load the embedding files
import pickle
with open('/content/drive/MyDrive/CSCI6364Proj/embeddingscomp2VF.pkl', "rb") as fIn:
    stored_data = pickle.load(fIn)
    stored_sentences = stored_data['sentences']
    stored_embeddings = stored_data['embeddings']

In [13]:
#calculate similarity
from sklearn.metrics.pairwise import cosine_similarity
ans = cosine_similarity([tok],stored_embeddings)

In [14]:
#getting three most similar songs
import heapq
import csv
sim_list = ans[0]
#loading all available track id's
file = open("/content/drive/MyDrive/CSCI6364Proj/t_list.csv", "r")
data = list(csv.reader(file, delimiter=","))
t_list = data[0]
file.close()
result = heapq.nlargest(3, zip(sim_list,t_list))

In [15]:
result

[(0.78255653, 'TRMWKZD128F425A2F6'),
 (0.74746364, 'TRCWOZX12903CC84D1'),
 (0.73515254, 'TRXKILT128F9332E60')]

In [16]:
prim_seq = []
for i in range(len(result)):
  prim_seq.append(result[i][1])
print(prim_seq)

['TRMWKZD128F425A2F6', 'TRCWOZX12903CC84D1', 'TRXKILT128F9332E60']
